<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_11_CNN_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/xxx"

# 第11回 Convolutional Neural Networkの実装

## 11.1 DataLoaderの設定

データを効率的にロードし、ミニバッチ（小さなデータの塊）として提供するためのユーティリティ関数が揃っている

*   データの準備、Tensor型への変換など
*   データの作成、前処理など

In [ ]:
import torch
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer",
                   "dog", "frog", "horse", "ship", "truck"]

In [ ]:
# データ拡張（後で詳しく行う）
# affine = transforms.RandomAffine((-30, 30), scale=(0.8, 1.2))  # 回転とリサイズ
# flip = transforms.RandomHorizontalFlip(p=0.5)  # 左右反転

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))  # 平均値を0、標準偏差を1に

# 前処理を順に記述したリストを入力
transform_train = transforms.Compose( ***here*** )
transform_test = transforms.Compose( ***here*** )

In [ ]:
dataset_dirname = "./data"

# 学習データとテストデータの取得
cifar10_train = CIFAR10(dataset_dirname, train=True, download=True, transform=transform_train)
cifar10_test = CIFAR10(dataset_dirname, train=False, download=True, transform=transform_test)

# DataLoaderの設定
batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=batch_size, shuffle=False)

## 11.2 データセットの確認

In [ ]:
import matplotlib.pyplot as plt

n_img = 10  # 表示する画像の数
plt.figure(figsize=(10, 4))
for i in range(n_img):
    ax = plt.subplot(2, 5, i+1)
    ax.imshow(cifar10_train.data[i])
    ax.set_title(cifar10_classes[cifar10_train.targets[i]])
    ax.get_xaxis().set_visible(False)  # 軸を非表示に
    ax.get_yaxis().set_visible(False)
plt.show()

print("データの形状:", cifar10_train.data.shape)
print("ラベル値:", cifar10_train.targets[:n_img])

## 11.3 CNNモデルの構築

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)   # 畳み込み層:(入力チャンネル数, フィルタ数、フィルタサイズ)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)    # プーリング層:（領域のサイズ, 領域の間隔）
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 256) # 全結合層
        self.dropout = nn.Dropout(p=0.5)  # ドロップアウト:(p=ドロップアウト率)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16*5*5)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()
net.cuda()  # GPU対応
print(net)

## 11.4 CNNモデルの訓練

In [ ]:
from torch import optim

# 損失関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters())

# 損失のログ
record_loss_train = []
record_loss_test = []

# 学習
for i in range(10):  # 10エポック学習
    net.train()  # 訓練モード
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):  # ミニバッチ（x, t）を取り出す
        x, t = x.cuda(), t.cuda()  # GPU対応
        ***here*** # 予測計算
        ***here*** # 損失計算
        loss_train += loss.item()
        optimizer.zero_grad()
        ***here*** # 勾配計算
        ***here*** # パラメータ更新
    loss_train /= j+1
    record_loss_train.append(loss_train)

    net.eval()  # 評価モード
    loss_test = 0
    for j, (x, t) in enumerate(test_loader):  # ミニバッチ（x, t）を取り出す
        x, t = x.cuda(), t.cuda()
        y = net(x)
        loss = loss_fnc(y, t)
        loss_test += loss.item()
    loss_test /= j+1
    record_loss_test.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## 11.5 誤差の推移

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## 11.5 正解率の計算

In [ ]:
correct = 0
total = 0
net.eval()  # 評価モード
for i, (x, t) in enumerate(test_loader):
    x, t = x.cuda(), t.cuda()  # GPU対応
    y = net(x)
    correct += ***here***
    total += len(x)
print("正解率:", str(correct/total*100) + "%")

## 11.6 訓練済みモデルを使った予測

In [ ]:
cifar10_loader = DataLoader(cifar10_test, batch_size=1, shuffle=True)
dataiter = iter(cifar10_loader)
images, labels = next(dataiter)  # <--- サンプルを1つだけ取り出す

plt.imshow(images[0].permute(1, 2, 0))  # チャンネルを一番後ろに
plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # 図のラベルとメモリを非表示に
plt.show()

net.eval()  # 評価モード
x, t = images.cuda(), labels.cuda()  # GPU対応
y = net(x)
print("正解:", cifar10_classes[labels[0]],
      "予測結果:", cifar10_classes[y.argmax().item()])

## 11.7 モデルの保存と読み込み

学習済みモデルを保存しておいて、別のマシンで使うことが出来る。

または、学習済みモデルを読み込んで、追加データを用いての再学習を行うことが出来る。

In [ ]:
model_name = 'mymodel_1.pth'

# Torch Script形式で保存
model_scripted = torch.jit.script(net)
model_scripted.save('data/' + model_name)

In [ ]:
net = torch.jit.load('data/' + model_name)